In [28]:
!pip install opencv-python Pillow
!pip install opencv-python


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import numpy as np
from scipy.ndimage import gaussian_filter

class FoveateVision:

    def __init__(self, fovy, sigma=1):
        self._cam_eye_fovy = fovy
        self._sigma = sigma

    def foveate(self, img, foveal_center=None):
        """
        Foveate the image, note that the shape of image has to be (height, width, 3)
        """

        # Define the foveal region
        fov = self._cam_eye_fovy
        foveal_size = 15
        foveal_pixels = int(foveal_size / 2 * img.shape[0] / fov)

        if foveal_center is None:  # default to center if not provided
            foveal_center = (img.shape[0] // 2, img.shape[1] // 2)

        # Create a circular mask for the foveal region
        y, x = np.ogrid[-foveal_center[0]:img.shape[0] - foveal_center[0], -foveal_center[1]:img.shape[1] - foveal_center[1]]
        mask = x ** 2 + y ** 2 <= (foveal_pixels ** 2)

        # Apply a Gaussian blur to each color channel separately
        blurred = np.zeros_like(img)
        for c in range(3):
            blurred_channel = gaussian_filter(img[:, :, c], sigma=self._sigma)
            blurred[:, :, c][~mask] = blurred_channel[~mask]

        # Combine the original image and the blurred image
        foveated = img.copy()
        foveated[~mask] = blurred[~mask]

        return foveated

In [52]:
import cv2

# Load the image using OpenCV
img_cv = cv2.imread('subfigure3.jpg')

# Convert the color space from BGR (default in OpenCV) to RGB
img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)

# Now, img_rgb is your image array ready for processing
img = img_rgb # your numpy image array here

processor = FoveateVision(fovy=50, sigma=5)
result = processor.foveate(img, foveal_center=(300, 600))

# Convert the result back to BGR color space for saving with OpenCV
result_bgr = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)

# Save the result using OpenCV
cv2.imwrite('vision3.jpg', result_bgr)

True